In [15]:
import pandas as pd
import numpy as np
from IPython.display import display
from fastai.imports import *
from sklearn import metrics
from pandas.tseries.offsets import DateOffset
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr


hour_ahead = 24



path = "/content/drive/MyDrive/Colab Notebooks/Project/01_weather_FI.csv"
Merge_all = pd.read_csv(path)
dti = pd.to_datetime(Merge_all['DateTime'], format='%Y-%m-%d %H', exact=False)
Merge_all.set_index(pd.Index(dti), inplace=True)
Merge_all.drop(['DateTime'], axis=1, inplace=True)

First = Merge_all.iloc[0:11000, :]
N_index = First.shape
Y_temp = First['FI']
Y = Y_temp.iloc[hour_ahead:N_index[0]]
#Y = Y.to_numpy()
#Y.reset_index(drop=True, inplace=True)
#Y.columns = [''] * len(Y.columns)


X_temp1 = First['FI']
X_temp2 = First.loc[:, First.columns != 'FI']
#X_temp1 = First.drop(First.columns.difference(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell']), axis=1)
#X_temp2 = First.drop(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell'], axis=1)
X_temp2_index = X_temp2.index + DateOffset(hours=-hour_ahead)
X_temp2.set_index(X_temp2_index, inplace=True)
X_temp1 = X_temp1.to_frame()
X_temp = X_temp1.join(X_temp2)
X = X_temp.iloc[0:N_index[0] - hour_ahead]
#X = X.to_numpy()
#X.reset_index(drop=True, inplace=True)
#X.columns = [''] * len(X.columns)
Pred_index = X.shape
print("Shape = "+str(Pred_index))


CORR_index = []
CORR = []
num_cols = X.columns
for i in num_cols:
    corr, _ = pearsonr(X[i], Y)
    CORR_index.append(i)
    CORR.append(corr)

      
Yci = pd.DataFrame({'Index':CORR_index})
Yc = pd.DataFrame({'Y':CORR})
Yc.index = Yci['Index']
Yc.sort_values(by=['Y'], inplace=True)
print(Yc)

Shape = (10976, 780)
                                      Y
Index                                  
65_Relative humidity (%)      -0.218136
156_Relative humidity (%)     -0.217913
68_Relative humidity (%)      -0.214076
77_Relative humidity (%)      -0.211218
97_Relative humidity (%)      -0.210638
50_Relative humidity (%)      -0.210188
141_Relative humidity (%)     -0.209585
51_Relative humidity (%)      -0.204845
23_Relative humidity (%)      -0.202247
136_Relative humidity (%)     -0.199226
14_Relative humidity (%)      -0.197094
92_Relative humidity (%)      -0.195960
143_Relative humidity (%)     -0.194483
64_Relative humidity (%)      -0.192635
54_Relative humidity (%)      -0.191490
157_Relative humidity (%)     -0.190895
159_Relative humidity (%)     -0.190158
41_Relative humidity (%)      -0.188458
70_Relative humidity (%)      -0.188435
72_Relative humidity (%)      -0.187707
168_Relative humidity (%)     -0.183906
42_Relative humidity (%)      -0.183268
16_Relative humidit

In [16]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [17]:
Y_temp = Y.copy()
Y_temp.index = Y_temp.index + DateOffset(hours=-hour_ahead)
Y_temp.rename('Y', inplace=True)
XY = X.copy()
XY = XY.join(Y_temp)
XYC = XY.corr()

In [18]:
YCorr = XYC.drop(First.columns.difference(['Y']), axis=1)
YCorr.sort_values(by=['Y'], inplace=True, ascending=False)
print(YCorr)

                                      Y
Y                              1.000000
FI                             0.676127
48_Wind direction (deg)        0.058847
9_Wind direction (deg)         0.058281
172_Wind direction (deg)       0.052917
17_Wind direction (deg)        0.052492
10_Wind direction (deg)        0.052273
28_Wind direction (deg)        0.050181
121_Wind direction (deg)       0.043991
95_Wind direction (deg)        0.043405
85_Wind direction (deg)        0.043274
58_Wind direction (deg)        0.042358
87_Wind direction (deg)        0.040576
59_Wind direction (deg)        0.040330
101_Wind direction (deg)       0.034521
12_Wind direction (deg)        0.034172
61_Wind direction (deg)        0.033324
133_Wind direction (deg)       0.031872
116_Wind direction (deg)       0.031484
50_Wind direction (deg)        0.027444
153_Wind direction (deg)       0.026998
141_Wind direction (deg)       0.026750
19_Wind direction (deg)        0.026012
39_Wind direction (deg)        0.025509


In [19]:
YCorr = XYC.drop(First.columns.difference(['Y']), axis=1)
YCorr.sort_values(by=['Y'], inplace=True, ascending=True)
print(YCorr)

                                      Y
65_Relative humidity (%)      -0.218164
156_Relative humidity (%)     -0.217702
77_Relative humidity (%)      -0.213918
68_Relative humidity (%)      -0.212641
97_Relative humidity (%)      -0.212031
50_Relative humidity (%)      -0.211571
141_Relative humidity (%)     -0.211093
51_Relative humidity (%)      -0.205331
23_Relative humidity (%)      -0.203801
136_Relative humidity (%)     -0.199661
143_Relative humidity (%)     -0.196311
14_Relative humidity (%)      -0.195095
92_Relative humidity (%)      -0.194913
64_Relative humidity (%)      -0.194755
54_Relative humidity (%)      -0.193350
159_Relative humidity (%)     -0.192363
157_Relative humidity (%)     -0.191901
41_Relative humidity (%)      -0.189511
72_Relative humidity (%)      -0.188877
70_Relative humidity (%)      -0.188045
168_Relative humidity (%)     -0.184275
42_Relative humidity (%)      -0.184203
79_Relative humidity (%)      -0.182958
16_Relative humidity (%)      -0.182300


In [20]:
YCorrABS = YCorr.abs()
YCorrABS.sort_values(by=['Y'], inplace=True, ascending=False)
print(YCorrABS)

                                      Y
Y                              1.000000
FI                             0.676127
65_Relative humidity (%)       0.218164
156_Relative humidity (%)      0.217702
77_Relative humidity (%)       0.213918
68_Relative humidity (%)       0.212641
97_Relative humidity (%)       0.212031
50_Relative humidity (%)       0.211571
141_Relative humidity (%)      0.211093
51_Relative humidity (%)       0.205331
23_Relative humidity (%)       0.203801
136_Relative humidity (%)      0.199661
143_Relative humidity (%)      0.196311
14_Relative humidity (%)       0.195095
92_Relative humidity (%)       0.194913
64_Relative humidity (%)       0.194755
54_Relative humidity (%)       0.193350
159_Relative humidity (%)      0.192363
157_Relative humidity (%)      0.191901
41_Relative humidity (%)       0.189511
72_Relative humidity (%)       0.188877
70_Relative humidity (%)       0.188045
168_Relative humidity (%)      0.184275
42_Relative humidity (%)       0.184203


In [21]:
XYC2 = XY.corr(method='spearman')
YCorr2 = XYC2.drop(First.columns.difference(['Y']), axis=1)
YCorr2.sort_values(by=['Y'], inplace=True, ascending=False)
print(YCorr2)

                                      Y
Y                              1.000000
FI                             0.771028
48_Wind direction (deg)        0.094772
9_Wind direction (deg)         0.093368
172_Wind direction (deg)       0.086895
95_Wind direction (deg)        0.084781
85_Wind direction (deg)        0.084619
17_Wind direction (deg)        0.080208
10_Wind direction (deg)        0.079907
87_Wind direction (deg)        0.079008
59_Wind direction (deg)        0.078951
28_Wind direction (deg)        0.076082
58_Wind direction (deg)        0.073268
43_Wind direction (deg)        0.071635
101_Wind direction (deg)       0.060763
50_Wind direction (deg)        0.058582
39_Wind direction (deg)        0.057663
141_Wind direction (deg)       0.057592
153_Wind direction (deg)       0.057572
121_Wind direction (deg)       0.053815
133_Wind direction (deg)       0.053160
12_Wind direction (deg)        0.050439
19_Wind direction (deg)        0.046891
5_Wind direction (deg)         0.043311


In [22]:
XYC2 = XY.corr(method='spearman')
YCorr2 = XYC2.drop(First.columns.difference(['Y']), axis=1)
YCorr2.sort_values(by=['Y'], inplace=True, ascending=True)
print(YCorr2)

                                      Y
64_Relative humidity (%)      -0.288611
141_Relative humidity (%)     -0.274917
50_Relative humidity (%)      -0.273867
103_Relative humidity (%)     -0.259493
51_Relative humidity (%)      -0.258334
72_Relative humidity (%)      -0.256871
48_Relative humidity (%)      -0.255143
12_Relative humidity (%)      -0.253337
2_Relative humidity (%)       -0.251145
17_Relative humidity (%)      -0.249553
157_Relative humidity (%)     -0.248659
10_Relative humidity (%)      -0.248426
148_Relative humidity (%)     -0.246215
63_Relative humidity (%)      -0.244844
6_Relative humidity (%)       -0.243424
41_Relative humidity (%)      -0.242415
36_Relative humidity (%)      -0.242009
5_Relative humidity (%)       -0.241965
133_Relative humidity (%)     -0.241894
14_Relative humidity (%)      -0.241606
171_Relative humidity (%)     -0.239198
116_Relative humidity (%)     -0.239173
91_Relative humidity (%)      -0.238299
68_Relative humidity (%)      -0.237731


In [23]:
YCorr2ABS = YCorr2.abs()
YCorr2ABS.sort_values(by=['Y'], inplace=True, ascending=False)
print(YCorr2ABS)

                                      Y
Y                              1.000000
FI                             0.771028
64_Relative humidity (%)       0.288611
141_Relative humidity (%)      0.274917
50_Relative humidity (%)       0.273867
103_Relative humidity (%)      0.259493
51_Relative humidity (%)       0.258334
72_Relative humidity (%)       0.256871
48_Relative humidity (%)       0.255143
12_Relative humidity (%)       0.253337
2_Relative humidity (%)        0.251145
17_Relative humidity (%)       0.249553
157_Relative humidity (%)      0.248659
10_Relative humidity (%)       0.248426
148_Relative humidity (%)      0.246215
63_Relative humidity (%)       0.244844
6_Relative humidity (%)        0.243424
41_Relative humidity (%)       0.242415
36_Relative humidity (%)       0.242009
5_Relative humidity (%)        0.241965
133_Relative humidity (%)      0.241894
14_Relative humidity (%)       0.241606
171_Relative humidity (%)      0.239198
116_Relative humidity (%)      0.239173


In [24]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X, Y)
importance = model.feature_importances_
Feature = []
Feature_index = X.columns
for i, v in enumerate(importance):
    Feature.append(v)

Yfi = pd.DataFrame({'Index':Feature_index})
Yf = pd.DataFrame({'Y':Feature})
Yf.index = Yfi['Index']
Yf.sort_values(by=['Y'], inplace=True, ascending=False)
print(Yf)

[15:37:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
                                      Y
Index                                  
FI                             0.036824
32_Relative humidity (%)       0.029505
43_Relative humidity (%)       0.025106
97_Wind direction (deg)        0.020725
2_Relative humidity (%)        0.018168
51_Wind direction (deg)        0.017108
72_Wind direction (deg)        0.013282
58_Air temperature (degC)      0.011884
172_Wind speed (m/s)           0.011500
142_Relative humidity (%)      0.011270
15_Relative humidity (%)       0.011162
15_Wind speed (m/s)            0.011093
8_Air temperature (degC)       0.010607
133_Relative humidity (%)      0.009657
165_Relative humidity (%)      0.009634
24_Wind direction (deg)        0.009468
159_Air temperature (degC)     0.009358
116_Relative humidity (%)      0.009031
64_Air temperature (degC)      0.009023
129_Air temperature (degC)     0.

In [25]:
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
model2.fit(X, Y)
importance2 = model2.coef_
FeatureR = []
FeatureR_index = X.columns
for i,v in enumerate(importance2):
    FeatureR.append(v)

Yfri = pd.DataFrame({'Index':FeatureR_index})
Yfr = pd.DataFrame({'Y':FeatureR})
Yfr.index = Yfri['Index']
Yfr.sort_values(by=['Y'], inplace=True, ascending=False)
print(Yfr)

                                      Y
Index                                  
FI                             0.548037
134_Wind speed (m/s)           0.239734
149_Air temperature (degC)     0.202078
13_Air temperature (degC)      0.187788
65_Air temperature (degC)      0.186191
110_Precipitation amount (mm)  0.184494
101_Wind speed (m/s)           0.182851
1_Air temperature (degC)       0.178736
19_Precipitation amount (mm)   0.178036
36_Precipitation amount (mm)   0.176435
131_Precipitation amount (mm)  0.166925
97_Air temperature (degC)      0.165160
68_Air temperature (degC)      0.165113
22_Air temperature (degC)      0.163531
163_Air temperature (degC)     0.157210
154_Air temperature (degC)     0.155502
124_Air temperature (degC)     0.153979
77_Wind speed (m/s)            0.152162
170_Cloud amount (1/8)         0.149728
72_Wind speed (m/s)            0.147911
96_Air temperature (degC)      0.145201
105_Relative humidity (%)      0.141193
30_Precipitation amount (mm)   0.139928


In [26]:
Yfri = pd.DataFrame({'Index':FeatureR_index})
Yfr = pd.DataFrame({'Y':FeatureR})
Yfr.index = Yfri['Index']
Yfr.sort_values(by=['Y'], inplace=True, ascending=True)
print(Yfr)

                                      Y
Index                                  
4_Precipitation amount (mm)   -0.212882
100_Cloud amount (1/8)        -0.191183
125_Precipitation amount (mm) -0.186613
12_Precipitation amount (mm)  -0.185029
162_Precipitation amount (mm) -0.183618
94_Air temperature (degC)     -0.182597
57_Wind speed (m/s)           -0.182541
93_Air temperature (degC)     -0.179633
159_Precipitation amount (mm) -0.176173
15_Wind speed (m/s)           -0.173817
53_Wind speed (m/s)           -0.172058
138_Air temperature (degC)    -0.166167
1_Precipitation amount (mm)   -0.166028
75_Precipitation amount (mm)  -0.163619
77_Air temperature (degC)     -0.161893
123_Wind speed (m/s)          -0.161575
38_Cloud amount (1/8)         -0.160607
125_Air temperature (degC)    -0.151030
137_Wind speed (m/s)          -0.149219
151_Wind speed (m/s)          -0.146410
33_Air temperature (degC)     -0.145151
135_Precipitation amount (mm) -0.144583
174_Air temperature (degC)    -0.144180


In [27]:
YfrABS = Yfr.abs()
YfrABS.sort_values(by=['Y'], inplace=True, ascending=False)
print(YfrABS)

                                      Y
Index                                  
FI                             0.548037
134_Wind speed (m/s)           0.239734
4_Precipitation amount (mm)    0.212882
149_Air temperature (degC)     0.202078
100_Cloud amount (1/8)         0.191183
13_Air temperature (degC)      0.187788
125_Precipitation amount (mm)  0.186613
65_Air temperature (degC)      0.186191
12_Precipitation amount (mm)   0.185029
110_Precipitation amount (mm)  0.184494
162_Precipitation amount (mm)  0.183618
101_Wind speed (m/s)           0.182851
94_Air temperature (degC)      0.182597
57_Wind speed (m/s)            0.182541
93_Air temperature (degC)      0.179633
1_Air temperature (degC)       0.178736
19_Precipitation amount (mm)   0.178036
36_Precipitation amount (mm)   0.176435
159_Precipitation amount (mm)  0.176173
15_Wind speed (m/s)            0.173817
53_Wind speed (m/s)            0.172058
131_Precipitation amount (mm)  0.166925
138_Air temperature (degC)     0.166167


In [28]:
import pandas as pd
import numpy as np
from IPython.display import display
from fastai.imports import *
from sklearn import metrics
from pandas.tseries.offsets import DateOffset
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr


hour_ahead = 24



path = "/content/drive/MyDrive/Colab Notebooks/Project/06_Weather_FI_Time.csv"
Merge_all = pd.read_csv(path)
dti = pd.to_datetime(Merge_all['DateTime'], format='%Y-%m-%d %H', exact=False)
Merge_all.set_index(pd.Index(dti), inplace=True)
Merge_all.drop(['DateTime'], axis=1, inplace=True)

First = Merge_all.iloc[0:20000, :]
N_index = First.shape
Y_temp = First['FI']
Y = Y_temp.iloc[hour_ahead:N_index[0]]
Y = Y.iloc[0:11000]
#Y = Y.to_numpy()


X_temp1 = First['FI']
X_temp2 = First.loc[:, First.columns != 'FI']
#X_temp1 = First.drop(First.columns.difference(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell']), axis=1)
#X_temp2 = First.drop(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell'], axis=1)
X_temp2_index = X_temp2.index + DateOffset(hours=-hour_ahead)
X_temp2.set_index(X_temp2_index, inplace=True)
X_temp1 = X_temp1.to_frame()
X_temp = X_temp1.join(X_temp2)
X = X_temp.iloc[0:N_index[0] - hour_ahead]
X = X.iloc[0:11000]
#X = X.to_numpy()

Pred_index = X.shape
print("Shape = "+str(Pred_index))
print(X.head(10))
print(Y.shape)
print(Y.head(10))

Shape = (11000, 789)
                        FI  ...  Parts_of_the_Day
DateTime                    ...                  
2013-01-01 00:00:00  31.04  ...               3.0
2013-01-01 01:00:00  27.51  ...               3.0
2013-01-01 02:00:00  24.44  ...               3.0
2013-01-01 03:00:00  21.81  ...               3.0
2013-01-01 04:00:00  22.37  ...               3.0
2013-01-01 05:00:00  25.51  ...               3.0
2013-01-01 06:00:00  27.93  ...               0.0
2013-01-01 07:00:00  28.01  ...               0.0
2013-01-01 08:00:00  27.80  ...               0.0
2013-01-01 09:00:00  27.35  ...               0.0

[10 rows x 789 columns]
(11000,)
DateTime
2013-01-02 00:00:00    32.84
2013-01-02 01:00:00    32.35
2013-01-02 02:00:00    31.83
2013-01-02 03:00:00    32.17
2013-01-02 04:00:00    32.97
2013-01-02 05:00:00    35.02
2013-01-02 06:00:00    36.73
2013-01-02 07:00:00    42.50
2013-01-02 08:00:00    39.42
2013-01-02 09:00:00    38.54
Name: FI, dtype: float64
